# Step 0
## The REPL

In [1]:
def READ(program:str) -> str:
    read_str(program)
    return program

In [2]:
def EVAL(line:str) -> str:
    return line

In [111]:
def PRINT(line:str):
    pr_str(line)

In [4]:
def rep(program):
    PRINT(EVAL(READ(program)))

#### Tests

In [5]:
rep(';=>%')

;=>%


# Step 1

In [58]:
# Mal types?

Symbol = str          
List   = list         
Number = (int, float)

#### Test

In [59]:
dgg = Number(9)
type(dgg)

TypeError: 'tuple' object is not callable

In [80]:
def read_str(program):
    t = tokenize(program)
    r = Reader(t)
    l = read_form(r)
    return l

In [61]:
class Reader:
    def __init__(self, tokens):
        self.tokens = tokens
        self.position = 0
        self.length = len(tokens)
    def nex(self):
        return current_token
    def peek(self):
        current_token = self.tokens[self.position]
        return current_token

In [62]:
import re

r = r'''[\s,]*(~@|[\[\]{}()'`~^@]|"(?:\\.|[^\\"])*"|;.*|[^\s\[\]{}('"`,;)]*)'''

def tokenize(program:str) -> list:
    tkns = re.split(r, program)
    tkns_list = []
    for t in tkns:
        if t != '':
            tkns_list.append(t) ### ESTO ESTA MAL
    return tkns_list

#### Tests

In [63]:
t = tokenize('( + 2 (* 3 4))')

t

['(', '+', '2', '(', '*', '3', '4', ')', ')']

tokenize

    [\s,]*: Matches any number of whitespaces or commas. This is not captured so it will be ignored and not tokenized. --- ANDA PERO ESTA MAL

    ~@: Captures the special two-characters ~@ (tokenized). ---- ANDA

    [\[\]{}()'`~^@]: Captures any special single character, one of []{}()'`~^@ (tokenized). ---- NO ANDA LOS PONE TODOS JUNTOS

    "(?:\\.|[^\\"])*"?: Starts capturing at a double-quote and stops at the next double-quote unless it was preceded by a backslash in which case it includes it until the next double-quote (tokenized). It will also match unbalanced strings (no ending double-quote) which should be reported as an error. ---- ANDA

    ;.*: Captures any sequence of characters starting with ; (tokenized). ---- ANDA

    [^\s\[\]{}('"`,;)]*: Captures a sequence of zero or more non special characters (e.g. symbols, numbers, "true", "false", and "nil") and is sort of the inverse of the one above that captures special characters (tokenized).


In [69]:
def read_form(tokens:Reader):
    tkn = tokens.peek()
    if tkn == '(':
        t = read_list(tokens)
    else: 
        t = read_atom(tkn)
    return t

# TODO falta lo q avanza al siguiente token
# TODO  The return value from read_form is a mal data type.

In [70]:
def read_list(tokens):
    result = []
    tokens.nex() # skips the "(" that brought us here
    tkn = tokens.peek()
    while tkn != ')':
        t = read_form(tokens)
        result.append(t)
        tokens.nex()
        tkn = tokens.peek()
    return result

# TODO poner algo que corte si te olvidas un parentesis de cierre 
# TODO si pones paren de cierre de mas, los ignora, corregir

This function will repeatedly call read_form with the Reader object until it encounters a ')' token (if it reach EOF before reading a ')' then that is an error). It accumulates the results into a List type. If your language does not have a sequential data type that can hold mal type values you may need to implement one (in types.qx). Note that read_list repeatedly calls read_form rather than read_atom. This mutually recursive definition between read_list and read_form is what allows lists to contain lists.


In [71]:
def read_atom(tok):
    try: return int(tok)
    except ValueError:
        try: return float(tok)
        except ValueError:
            return str(tok)

In [72]:
ghh = read_atom('s')


#### Tests

In [85]:
my_str = "1 2 (abc)(ab)"

result = read_str(my_str)

print(result)
pr_str(result)

1


TypeError: 'int' object is not iterable

In [105]:
def pr_str(token):
    l = []
    if type(token) == list:
        for i in token:
            l.append(str(i))
        l = ' '.join(l)
        l = '(' + l + ')'
        return l
    else:
        s = str(t)
        result.append(s)
    return result


In [110]:
def pr_str(token):
    l = []
    if type(token) == list:
        for i in token:
            l.append(pr_str(i))
        l = ' '.join(l)
        l = '(' + l + ')'
        return l
    else:
        s = str(token)
        return s


pr_str([4, 2, ['a']] )

'(4 2 (a))'